In [1]:
import torch
from torch import nn, optim
from torch.nn import functional as F
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
from torch.autograd import Variable
from sklearn.metrics import r2_score
from sklearn import preprocessing
#import scipy.io as sio
import numpy as np
import pandas as pd

In [2]:
from scipy.stats import pearsonr

# Import data

In [3]:
data_r=pd.read_csv('gdsc_gene_cell.csv')
label_r=pd.read_csv('gdsc_drug.csv')

In [4]:
label_jy=pd.read_csv("GDSClabel.csv",index_col=0)
data_jy=pd.read_csv("GDSCexpression.csv",index_col=0)

In [5]:
np.nanmax(label_r.values)

0.857826

In [6]:
np.nanmax(label_jy.values)

0.857826

In [7]:
label_r=label_r.fillna(1)
label_jy=label_jy.fillna(1)

# Your is gene-cell, mine is cell-gene

In [8]:
data_r.head()

,BxPC-3,KMOE-2,MFM-223,NUGC-3,OC-314,COLO-741,KARPAS-45,JAR,DU-4475,MONO-MAC-6,...,SF268,TK10,A2780,OVCAR-4.1,A375,KP-N-S19s,BC-3,Panc 08.13.1,EKVX.1,DMS-114.1
ENSG00000000003_at,10.898588,7.294736,5.116163,9.865415,9.853360,8.530792,8.176877,10.006412,5.477983,5.104350,...,8.338001,9.008451,9.239745,9.472819,7.968322,7.993171,5.322980,9.665734,9.364049,8.128921
ENSG00000000005_at,4.201139,4.336010,4.170541,4.069192,4.010374,4.017925,4.519250,4.278360,4.434947,4.403349,...,4.349856,4.121786,4.177905,4.227348,4.198914,4.359674,4.366423,4.166249,4.294723,4.467436
ENSG00000000419_at,11.556533,11.106639,11.614775,11.670086,11.057632,10.646966,10.878239,11.017995,10.723468,10.504204,...,11.245667,11.175300,11.485499,11.504745,11.485343,11.189191,9.702628,10.958903,10.247844,12.187419
ENSG00000000457_at,5.453730,6.005694,5.865642,6.224943,5.149241,5.121324,5.868385,5.735913,6.644952,5.664282,...,5.387504,5.961516,5.988397,5.699127,6.254533,6.575614,5.885715,5.281582,5.453070,5.657234
ENSG00000000460_at,5.579061,6.482126,6.111542,5.722082,5.086321,6.260783,6.737340,6.782542,5.710254,5.971068,...,5.814175,5.514232,6.760882,6.022538,6.368033,6.994632,5.069775,5.521107,5.123464,5.369075


In [9]:
data_jy.head()

,ENSG00000000003_at,ENSG00000000005_at,ENSG00000000419_at,ENSG00000000457_at,ENSG00000000460_at,ENSG00000000938_at,ENSG00000000971_at,ENSG00000001084_at,ENSG00000001167_at,ENSG00000001461_at,...,ENSG00000279767_at,ENSG00000279821_at,ENSG00000279863_at,ENSG00000280017_at,ENSG00000280099_at,ENSG00000280109_at,ENSG00000280178_at,ENSG00000280315_at,ENSG00000280316_at,ENSG00000280401_at
BxPC-3,10.898588,4.201139,11.556533,5.453730,5.579061,4.292183,9.163200,8.117461,5.324914,5.681355,...,4.659608,4.380693,3.885167,4.063791,5.940486,4.458145,4.417491,4.217877,4.280332,4.670091
KMOE-2,7.294736,4.336010,11.106639,6.005694,6.482126,4.313389,5.444548,7.525063,5.879200,5.289107,...,4.216687,4.299949,4.029517,4.265309,7.625721,4.771245,4.528459,4.454017,4.576398,4.563178
MFM-223,5.116163,4.170541,11.614775,5.865642,6.111542,4.169045,4.536259,7.122486,4.889235,5.602236,...,4.245438,4.221933,3.931852,3.819795,6.262250,4.408866,4.557418,3.879970,4.515965,4.264553
NUGC-3,9.865415,4.069192,11.670086,6.224943,5.722082,4.340082,4.722633,9.286627,5.565469,5.024771,...,4.269594,4.230069,3.864026,4.160386,6.236633,4.655936,4.541264,3.744077,4.446651,4.353334
OC-314,9.853360,4.010374,11.057632,5.149241,5.086321,4.298940,4.730210,8.372358,5.291822,5.069430,...,4.218052,4.382424,3.761106,4.061488,7.755052,4.499583,4.440562,3.962520,4.511420,4.305250


# Exist duplicated columns 

In [10]:
label_r.loc[:,["A704.1"]].head()

,A704.1
Erlotinib,1.000000
AICAR,0.089977
Camptothecin,0.289133
Vinblastine,0.079470
Cisplatin,0.019784


In [11]:
label_r.loc[:,["A704"]].head()

,A704
Erlotinib,1.000000
AICAR,0.089977
Camptothecin,0.289133
Vinblastine,0.079470
Cisplatin,0.019784


In [12]:
label_r.shape

(139, 789)

In [13]:
label_jy.loc["A704",]

,Erlotinib,AICAR,Camptothecin,Vinblastine,Cisplatin,Cytarabine,Docetaxel,Methotrexate,ATRA,Gefitinib,...,CMK,Pyrimethamine,JW-7-52-1,A-443654,GW843682X,MS-275,Parthenolide,MG-132,KIN001-135,TGX221
A704,1.0,0.089977,0.289133,0.07947,0.019784,0.124785,0.070528,0.003345,0.028893,0.00092,...,0.055617,1.0,0.309821,0.241788,0.041567,1.0,1.0,1.0,1.0,1.0
A704,1.0,0.089977,0.289133,0.07947,0.019784,0.124785,0.070528,0.003345,0.028893,0.00092,...,0.055617,1.0,0.309821,0.241788,0.041567,1.0,1.0,1.0,1.0,1.0


In [14]:
# There should be 139 unique drugs
print(len(set(label_jy.T.index)))
print(label_jy.shape)

139
(789, 139)


# I scaled the data row-wise

In [16]:
data = data_r.T
label = label_r.T.iloc[:,1]
scaler = preprocessing.StandardScaler(with_mean=True, with_std=True)
data = scaler.fit_transform(data)

In [17]:
print(np.std(data))
print(np.mean(data))

0.9999999999999999
1.7364322541407683e-17


In [18]:
data.shape

(789, 11833)

In [19]:
label_r.shape

(139, 789)

# Split test train

In [20]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(data, label, test_size=0.2, random_state=42)

In [21]:
# X_train=data[:,:607]
# X_test=data[:,607:]
# df_csv1 = pd.DataFrame(data=X_train)
# #X_train = df_csv1.T
# df_csv2 = pd.DataFrame(data=X_test)
# #X_test = df_csv2.T
# #X_train = preprocessing.StandardScaler().fit_transform(X_train)
# #X_test = preprocessing.StandardScaler().fit_transform(X_test)
# Y_train=label.iloc[1,:607]
# Y_test=label.iloc[1,607:]
print(data.shape)
print(label.shape)
print(X_train.shape, Y_train.shape)
print(X_test.shape, Y_test.shape)

(789, 11833)
(789,)
(631, 11833) (631,)
(158, 11833) (158,)


In [22]:
print(X_train.max())
print(X_train.min())

23.789515962677616
-17.27771827263068


In [23]:
# Define parameters
epochs = 500 #200,500,1000
dim_au_in = X_train.shape[1] #973
dim_au_out = 128 #8, 16, 32, 64, 128, 256,512
dim_dnn_in = dim_au_out
dim_dnn_out=1

In [24]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# Assuming that we are on a CUDA machine, this should print a CUDA device:
print(device)
torch.cuda.set_device(device)

cuda:0


In [25]:
trainData = torch.FloatTensor(X_train).to(device)
testData = torch.FloatTensor(X_test).to(device)
y = torch.FloatTensor(Y_train.values).to(device)
# construct TensorDataset
train_dataset = TensorDataset(trainData, trainData)
test_dataset = TensorDataset(testData, testData)
trainDataLoader1 = DataLoader(dataset=train_dataset, batch_size=200, shuffle=False)

# The model

In [26]:
# Construct autoencoder model

class AE(nn.Module):
    def __init__(self,dim_au_in,dim_au_out):
        super(AE, self).__init__()
        self.dim = dim_au_in
        self.fc1 = nn.Linear(dim_au_in, 512)
        self.bn1 = nn.BatchNorm2d(512)

        self.fc2 = nn.Linear(512, dim_au_out)
        self.fc3 = nn.Linear(dim_au_out, 512)
        self.bn2 = nn.BatchNorm2d(512)

        self.fc4 = nn.Linear(512, dim_au_in)            
                    
    def encode(self, x):
        h1 = F.relu(self.fc1(x))
        return self.fc2(h1)
       
    def decode(self, z):
        h3 = F.relu(self.fc3(z))
        return torch.sigmoid(self.fc4(h3))

    def forward(self, x):
        z = self.encode(x.view(-1, self.dim))
        #return self.decode(z), x
        return self.decode(z), z
    

In [27]:
# Initialization

#autoencoder = AE(dim=train_dataset.shape[1]).to(device)
autoencoder = AE(dim_au_in = X_train.shape[1],dim_au_out=dim_au_out).to(device)
optimizer = optim.Adam(autoencoder.parameters(), lr=1e-3)
#loss1
loss_func = nn.MSELoss().to(device)
#loss2
#loss_func = nn.BCELoss()
loss_train = np.zeros((epochs, 1))

In [28]:
# Train autoencoder

for epoch in range(epochs):
    # 不需要label，所以用一个占位符"_"代替
    for batchidx, (x, _) in enumerate(trainDataLoader1):
        x.requires_grad_(True)
        # encode and decode 
        decoded, encoded = autoencoder(x)
        # compute loss
        print(encoded.shape, decoded.shape)
        loss = loss_func(decoded, x)      
        # update
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
           
    loss_train[epoch,0] = loss.item()  
    print('Epoch: %04d, Training loss=%.8f' %
          (epoch+1, loss.item()))

torch.Size([200, 128]) torch.Size([200, 11833])
torch.Size([200, 128]) torch.Size([200, 11833])
torch.Size([200, 128]) torch.Size([200, 11833])
torch.Size([31, 128]) torch.Size([31, 11833])
Epoch: 0001, Training loss=1.07959545
torch.Size([200, 128]) torch.Size([200, 11833])
torch.Size([200, 128]) torch.Size([200, 11833])
torch.Size([200, 128]) torch.Size([200, 11833])
torch.Size([31, 128]) torch.Size([31, 11833])
Epoch: 0002, Training loss=0.99177527
torch.Size([200, 128]) torch.Size([200, 11833])
torch.Size([200, 128]) torch.Size([200, 11833])
torch.Size([200, 128]) torch.Size([200, 11833])
torch.Size([31, 128]) torch.Size([31, 11833])
Epoch: 0003, Training loss=0.96516711
torch.Size([200, 128]) torch.Size([200, 11833])
torch.Size([200, 128]) torch.Size([200, 11833])
torch.Size([200, 128]) torch.Size([200, 11833])
torch.Size([31, 128]) torch.Size([31, 11833])
Epoch: 0004, Training loss=0.94797027
torch.Size([200, 128]) torch.Size([200, 11833])
torch.Size([200, 128]) torch.Size([200, 

Epoch: 0036, Training loss=0.71257162
torch.Size([200, 128]) torch.Size([200, 11833])
torch.Size([200, 128]) torch.Size([200, 11833])
torch.Size([200, 128]) torch.Size([200, 11833])
torch.Size([31, 128]) torch.Size([31, 11833])
Epoch: 0037, Training loss=0.70999259
torch.Size([200, 128]) torch.Size([200, 11833])
torch.Size([200, 128]) torch.Size([200, 11833])
torch.Size([200, 128]) torch.Size([200, 11833])
torch.Size([31, 128]) torch.Size([31, 11833])
Epoch: 0038, Training loss=0.70805222
torch.Size([200, 128]) torch.Size([200, 11833])
torch.Size([200, 128]) torch.Size([200, 11833])
torch.Size([200, 128]) torch.Size([200, 11833])
torch.Size([31, 128]) torch.Size([31, 11833])
Epoch: 0039, Training loss=0.70618504
torch.Size([200, 128]) torch.Size([200, 11833])
torch.Size([200, 128]) torch.Size([200, 11833])
torch.Size([200, 128]) torch.Size([200, 11833])
torch.Size([31, 128]) torch.Size([31, 11833])
Epoch: 0040, Training loss=0.70455152
torch.Size([200, 128]) torch.Size([200, 11833])
to

torch.Size([31, 128]) torch.Size([31, 11833])
Epoch: 0072, Training loss=0.68714780
torch.Size([200, 128]) torch.Size([200, 11833])
torch.Size([200, 128]) torch.Size([200, 11833])
torch.Size([200, 128]) torch.Size([200, 11833])
torch.Size([31, 128]) torch.Size([31, 11833])
Epoch: 0073, Training loss=0.68696654
torch.Size([200, 128]) torch.Size([200, 11833])
torch.Size([200, 128]) torch.Size([200, 11833])
torch.Size([200, 128]) torch.Size([200, 11833])
torch.Size([31, 128]) torch.Size([31, 11833])
Epoch: 0074, Training loss=0.68666434
torch.Size([200, 128]) torch.Size([200, 11833])
torch.Size([200, 128]) torch.Size([200, 11833])
torch.Size([200, 128]) torch.Size([200, 11833])
torch.Size([31, 128]) torch.Size([31, 11833])
Epoch: 0075, Training loss=0.68700981
torch.Size([200, 128]) torch.Size([200, 11833])
torch.Size([200, 128]) torch.Size([200, 11833])
torch.Size([200, 128]) torch.Size([200, 11833])
torch.Size([31, 128]) torch.Size([31, 11833])
Epoch: 0076, Training loss=0.68745720
torc

torch.Size([200, 128]) torch.Size([200, 11833])
torch.Size([31, 128]) torch.Size([31, 11833])
Epoch: 0108, Training loss=0.68107033
torch.Size([200, 128]) torch.Size([200, 11833])
torch.Size([200, 128]) torch.Size([200, 11833])
torch.Size([200, 128]) torch.Size([200, 11833])
torch.Size([31, 128]) torch.Size([31, 11833])
Epoch: 0109, Training loss=0.68096983
torch.Size([200, 128]) torch.Size([200, 11833])
torch.Size([200, 128]) torch.Size([200, 11833])
torch.Size([200, 128]) torch.Size([200, 11833])
torch.Size([31, 128]) torch.Size([31, 11833])
Epoch: 0110, Training loss=0.68095005
torch.Size([200, 128]) torch.Size([200, 11833])
torch.Size([200, 128]) torch.Size([200, 11833])
torch.Size([200, 128]) torch.Size([200, 11833])
torch.Size([31, 128]) torch.Size([31, 11833])
Epoch: 0111, Training loss=0.68052125
torch.Size([200, 128]) torch.Size([200, 11833])
torch.Size([200, 128]) torch.Size([200, 11833])
torch.Size([200, 128]) torch.Size([200, 11833])
torch.Size([31, 128]) torch.Size([31, 11

torch.Size([200, 128]) torch.Size([200, 11833])
torch.Size([200, 128]) torch.Size([200, 11833])
torch.Size([31, 128]) torch.Size([31, 11833])
Epoch: 0144, Training loss=0.67541999
torch.Size([200, 128]) torch.Size([200, 11833])
torch.Size([200, 128]) torch.Size([200, 11833])
torch.Size([200, 128]) torch.Size([200, 11833])
torch.Size([31, 128]) torch.Size([31, 11833])
Epoch: 0145, Training loss=0.67584252
torch.Size([200, 128]) torch.Size([200, 11833])
torch.Size([200, 128]) torch.Size([200, 11833])
torch.Size([200, 128]) torch.Size([200, 11833])
torch.Size([31, 128]) torch.Size([31, 11833])
Epoch: 0146, Training loss=0.67490363
torch.Size([200, 128]) torch.Size([200, 11833])
torch.Size([200, 128]) torch.Size([200, 11833])
torch.Size([200, 128]) torch.Size([200, 11833])
torch.Size([31, 128]) torch.Size([31, 11833])
Epoch: 0147, Training loss=0.67508096
torch.Size([200, 128]) torch.Size([200, 11833])
torch.Size([200, 128]) torch.Size([200, 11833])
torch.Size([200, 128]) torch.Size([200, 

...

torch.Size([200, 128]) torch.Size([200, 11833])
torch.Size([200, 128]) torch.Size([200, 11833])
torch.Size([31, 128]) torch.Size([31, 11833])
Epoch: 0323, Training loss=0.65134835
torch.Size([200, 128]) torch.Size([200, 11833])
torch.Size([200, 128]) torch.Size([200, 11833])
torch.Size([200, 128]) torch.Size([200, 11833])
torch.Size([31, 128]) torch.Size([31, 11833])
Epoch: 0324, Training loss=0.65137661
torch.Size([200, 128]) torch.Size([200, 11833])
torch.Size([200, 128]) torch.Size([200, 11833])
torch.Size([200, 128]) torch.Size([200, 11833])
torch.Size([31, 128]) torch.Size([31, 11833])
Epoch: 0325, Training loss=0.65136540
torch.Size([200, 128]) torch.Size([200, 11833])
torch.Size([200, 128]) torch.Size([200, 11833])
torch.Size([200, 128]) torch.Size([200, 11833])
torch.Size([31, 128]) torch.Size([31, 11833])
Epoch: 0326, Training loss=0.65134329
torch.Size([200, 128]) torch.Size([200, 11833])
torch.Size([200, 128]) torch.Size([200, 11833])
torch.Size([200, 128]) torch.Size([200, 

torch.Size([200, 128]) torch.Size([200, 11833])
torch.Size([200, 128]) torch.Size([200, 11833])
torch.Size([200, 128]) torch.Size([200, 11833])
torch.Size([31, 128]) torch.Size([31, 11833])
Epoch: 0359, Training loss=0.64837557
torch.Size([200, 128]) torch.Size([200, 11833])
torch.Size([200, 128]) torch.Size([200, 11833])
torch.Size([200, 128]) torch.Size([200, 11833])
torch.Size([31, 128]) torch.Size([31, 11833])
Epoch: 0360, Training loss=0.64805955
torch.Size([200, 128]) torch.Size([200, 11833])
torch.Size([200, 128]) torch.Size([200, 11833])
torch.Size([200, 128]) torch.Size([200, 11833])
torch.Size([31, 128]) torch.Size([31, 11833])
Epoch: 0361, Training loss=0.64779127
torch.Size([200, 128]) torch.Size([200, 11833])
torch.Size([200, 128]) torch.Size([200, 11833])
torch.Size([200, 128]) torch.Size([200, 11833])
torch.Size([31, 128]) torch.Size([31, 11833])
Epoch: 0362, Training loss=0.64819592
torch.Size([200, 128]) torch.Size([200, 11833])
torch.Size([200, 128]) torch.Size([200, 

Epoch: 0394, Training loss=0.64499247
torch.Size([200, 128]) torch.Size([200, 11833])
torch.Size([200, 128]) torch.Size([200, 11833])
torch.Size([200, 128]) torch.Size([200, 11833])
torch.Size([31, 128]) torch.Size([31, 11833])
Epoch: 0395, Training loss=0.64506668
torch.Size([200, 128]) torch.Size([200, 11833])
torch.Size([200, 128]) torch.Size([200, 11833])
torch.Size([200, 128]) torch.Size([200, 11833])
torch.Size([31, 128]) torch.Size([31, 11833])
Epoch: 0396, Training loss=0.64521164
torch.Size([200, 128]) torch.Size([200, 11833])
torch.Size([200, 128]) torch.Size([200, 11833])
torch.Size([200, 128]) torch.Size([200, 11833])
torch.Size([31, 128]) torch.Size([31, 11833])
Epoch: 0397, Training loss=0.64478689
torch.Size([200, 128]) torch.Size([200, 11833])
torch.Size([200, 128]) torch.Size([200, 11833])
torch.Size([200, 128]) torch.Size([200, 11833])
torch.Size([31, 128]) torch.Size([31, 11833])
Epoch: 0398, Training loss=0.64491534
torch.Size([200, 128]) torch.Size([200, 11833])
to

torch.Size([31, 128]) torch.Size([31, 11833])
Epoch: 0430, Training loss=0.64255381
torch.Size([200, 128]) torch.Size([200, 11833])
torch.Size([200, 128]) torch.Size([200, 11833])
torch.Size([200, 128]) torch.Size([200, 11833])
torch.Size([31, 128]) torch.Size([31, 11833])
Epoch: 0431, Training loss=0.64257330
torch.Size([200, 128]) torch.Size([200, 11833])
torch.Size([200, 128]) torch.Size([200, 11833])
torch.Size([200, 128]) torch.Size([200, 11833])
torch.Size([31, 128]) torch.Size([31, 11833])
Epoch: 0432, Training loss=0.64260292
torch.Size([200, 128]) torch.Size([200, 11833])
torch.Size([200, 128]) torch.Size([200, 11833])
torch.Size([200, 128]) torch.Size([200, 11833])
torch.Size([31, 128]) torch.Size([31, 11833])
Epoch: 0433, Training loss=0.64244187
torch.Size([200, 128]) torch.Size([200, 11833])
torch.Size([200, 128]) torch.Size([200, 11833])
torch.Size([200, 128]) torch.Size([200, 11833])
torch.Size([31, 128]) torch.Size([31, 11833])
Epoch: 0434, Training loss=0.64228046
torc

torch.Size([200, 128]) torch.Size([200, 11833])
torch.Size([31, 128]) torch.Size([31, 11833])
Epoch: 0466, Training loss=0.64065099
torch.Size([200, 128]) torch.Size([200, 11833])
torch.Size([200, 128]) torch.Size([200, 11833])
torch.Size([200, 128]) torch.Size([200, 11833])
torch.Size([31, 128]) torch.Size([31, 11833])
Epoch: 0467, Training loss=0.64033794
torch.Size([200, 128]) torch.Size([200, 11833])
torch.Size([200, 128]) torch.Size([200, 11833])
torch.Size([200, 128]) torch.Size([200, 11833])
torch.Size([31, 128]) torch.Size([31, 11833])
Epoch: 0468, Training loss=0.64030629
torch.Size([200, 128]) torch.Size([200, 11833])
torch.Size([200, 128]) torch.Size([200, 11833])
torch.Size([200, 128]) torch.Size([200, 11833])
torch.Size([31, 128]) torch.Size([31, 11833])
Epoch: 0469, Training loss=0.64040607
torch.Size([200, 128]) torch.Size([200, 11833])
torch.Size([200, 128]) torch.Size([200, 11833])
torch.Size([200, 128]) torch.Size([200, 11833])
torch.Size([31, 128]) torch.Size([31, 11

In [29]:
# extract features
_, encodedTrainData = autoencoder(trainData)
featureTensor = encodedTrainData.double()
feature = featureTensor.detach().cpu().numpy()

In [30]:
print(feature.shape)

(631, 128)


In [51]:
feature.shape

(631, 128)

In [52]:
from sklearn import linear_model
clf = linear_model.Lasso(alpha=0.1)
clf.fit(feature, Y_train.values)

Lasso(alpha=0.1, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)

In [60]:
_,testFeature = autoencoder(testData)
lasso = clf.predict(testFeature.detach().cpu().numpy())

In [61]:
r2_score(lasso,Y_test)

-13.739109581715617

In [31]:
# Construct predictor

class DNN(nn.Module):
    def __init__(self,dim_dnn_in,dim_dnn_out):
        super(DNN, self).__init__()
        self.dim = dim_dnn_in
        #self.fc1 = nn.Linear(dim, 64)
        #self.fc2 = nn.Linear(64, 32)
        #self.fc3 = nn.Linear(32, 16)
        #self.fc4 = nn.Linear(16, num)
        self.fc1 = nn.Linear(dim_dnn_in, 128)
        self.fc2 = nn.Linear(128, 128)
        #self.fc3 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(128, dim_dnn_out)
        #self.fc4 = nn.Linear(64, num)
        #sigmoid=0/1
        self.sigmoid = torch.nn.Sigmoid()

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        #x = F.relu(self.fc3(x))
        y_hat=self.sigmoid(self.fc3(x))    
        #return self.decode(z), x
        return y_hat

In [32]:
EPOCH = 200

In [33]:
# Load data
# data type conversion
B_feature = torch.FloatTensor(feature).to(device)
y = torch.FloatTensor(Y_train.values).to(device)
# construct TensorDataset
b_data = TensorDataset(B_feature, y)
trainDataLoader2 = DataLoader(dataset=b_data, batch_size=200, shuffle=False)

In [34]:
b_data.tensors

(tensor([[ 12.4847,  -5.1559,  -2.6449,  ..., -31.8427,  -6.2808,  13.9576],
         [ 27.2028, -29.5773, -28.2250,  ..., -55.7418,  25.3317, -14.2840],
         [ 44.5594,  18.6824,  16.1929,  ...,   0.1352, -12.2825, -19.3472],
         ...,
         [-15.4021, -32.8844, -10.5443,  ...,   3.6307,  15.7637,  -0.5607],
         [ -2.9451,   1.2697,  -1.1724,  ...,   8.1977,   0.2348,   2.9714],
         [-20.3583,  11.0389, -16.0766,  ...,  -5.2953, -11.0096,  -4.1092]],
        device='cuda:0'),
 tensor([0.0851, 0.0447, 1.0000, 0.1016, 0.1935, 0.0314, 0.1041, 0.0852, 0.0183,
         1.0000, 1.0000, 0.2172, 0.1091, 0.1018, 0.2080, 0.0320, 0.0120, 0.2349,
         0.0925, 0.1001, 0.0035, 0.0572, 0.0514, 0.0043, 1.0000, 0.0299, 0.1407,
         0.1274, 0.1239, 0.1230, 1.0000, 1.0000, 0.0337, 0.0414, 0.0196, 0.0815,
         0.1463, 1.0000, 0.1764, 0.0986, 0.0300, 0.2797, 0.1579, 0.0035, 0.0970,
         0.1636, 0.0935, 0.2592, 1.0000, 1.0000, 0.1020, 0.0775, 0.0134, 0.0394,
         0.

In [35]:
b_data

In [36]:
dim_dnn_in

128

In [37]:
# Initialization DNN model

predictor = DNN(dim_dnn_in, dim_dnn_out).to(device)
optimizer = optim.Adam(predictor.parameters(), lr=1e-3,betas=(0.9,0.99))
#loss1-softmax
#loss_func = nn.MSELoss()
#loss2-sigmoid
loss_func = nn.BCELoss().to(device)
#loss3-sigmoid
#loss_func = nn.CrossEntropyLoss()

#criterion = torch.nn.MSELoss(size_average=True)
#criterion = torch.nn.BCELoss(size_average=True) # Defined loss function
#optimizer = optim.Adm(model.parameters(), lr=0.01) # Defined optimizer
loss_train = np.zeros((epochs, 1))

In [38]:
# train model
for epoch in range(EPOCH):
    print('Epoch: ',epoch)
    for step,(batch_x,batch_y) in enumerate(trainDataLoader2):
        b_x = Variable(batch_x)
        b_y = Variable(batch_y)
        # predict label
        output = predictor(b_x)
        # b_y=F.sigmoid(b_y) 
        
        #print
        #print(output)
        #print(b_y)
        # compute loss
        loss = loss_func(output,b_y)
        #loss = criterion(output, b_y)
        
        # update
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    loss_train[epoch,0] = loss.item()  
    print('Epoch: %04d, Training loss=%.8f' %
          (epoch+1, loss.item())) 


Epoch:  0


D:\Anaconda3\lib\site-packages\torch\nn\modules\loss.py:529: UserWarning: Using a target size (torch.Size([200])) that is different to the input size (torch.Size([200, 1])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)
D:\Anaconda3\lib\site-packages\torch\nn\modules\loss.py:529: UserWarning: Using a target size (torch.Size([31])) that is different to the input size (torch.Size([31, 1])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)


Epoch: 0001, Training loss=0.60170299
Epoch:  1
Epoch: 0002, Training loss=0.54621214
Epoch:  2
Epoch: 0003, Training loss=0.53574234
Epoch:  3
Epoch: 0004, Training loss=0.56248194
Epoch:  4
Epoch: 0005, Training loss=0.56033313
Epoch:  5
Epoch: 0006, Training loss=0.56515402
Epoch:  6
Epoch: 0007, Training loss=0.56323701
Epoch:  7
Epoch: 0008, Training loss=0.54850900
Epoch:  8
Epoch: 0009, Training loss=0.54357845
Epoch:  9
Epoch: 0010, Training loss=0.53810096
Epoch:  10
Epoch: 0011, Training loss=0.53549796
Epoch:  11
Epoch: 0012, Training loss=0.53699011
Epoch:  12
Epoch: 0013, Training loss=0.53593802
Epoch:  13
Epoch: 0014, Training loss=0.53486508
Epoch:  14
Epoch: 0015, Training loss=0.53172553
Epoch:  15
Epoch: 0016, Training loss=0.53005278
Epoch:  16
Epoch: 0017, Training loss=0.52945560
Epoch:  17
Epoch: 0018, Training loss=0.52940208
Epoch:  18
Epoch: 0019, Training loss=0.52806729
Epoch:  19
Epoch: 0020, Training loss=0.52723277
Epoch:  20
Epoch: 0021, Training loss=0.

Epoch: 0167, Training loss=0.52792627
Epoch:  167
Epoch: 0168, Training loss=0.52792579
Epoch:  168
Epoch: 0169, Training loss=0.52792615
Epoch:  169
Epoch: 0170, Training loss=0.52792531
Epoch:  170
Epoch: 0171, Training loss=0.52792650
Epoch:  171
Epoch: 0172, Training loss=0.52792531
Epoch:  172
Epoch: 0173, Training loss=0.52792770
Epoch:  173
Epoch: 0174, Training loss=0.52792543
Epoch:  174
Epoch: 0175, Training loss=0.52792925
Epoch:  175
Epoch: 0176, Training loss=0.52792478
Epoch:  176
Epoch: 0177, Training loss=0.52793109
Epoch:  177
Epoch: 0178, Training loss=0.52792281
Epoch:  178
Epoch: 0179, Training loss=0.52793342
Epoch:  179
Epoch: 0180, Training loss=0.52792025
Epoch:  180
Epoch: 0181, Training loss=0.52793747
Epoch:  181
Epoch: 0182, Training loss=0.52791709
Epoch:  182
Epoch: 0183, Training loss=0.52794069
Epoch:  183
Epoch: 0184, Training loss=0.52791417
Epoch:  184
Epoch: 0185, Training loss=0.52794081
Epoch:  185
Epoch: 0186, Training loss=0.52791560
Epoch:  186


In [39]:
# Get tesing feature
_,testFeature = autoencoder(testData)

In [40]:
testpredict = predictor(testFeature)

In [47]:
testpredict

tensor([[0.2517],
        [0.2476],
        [0.2988],
        [0.3000],
        [0.3214],
        [0.2434],
        [0.3187],
        [0.2279],
        [0.3155],
        [0.3317],
        [0.2544],
        [0.3380],
        [0.2455],
        [0.2857],
        [0.3438],
        [0.2998],
        [0.3015],
        [0.3026],
        [0.2920],
        [0.3410],
        [0.3099],
        [0.2036],
        [0.3496],
        [0.2456],
        [0.2859],
        [0.3059],
        [0.2764],
        [0.2419],
        [0.4340],
        [0.2387],
        [0.3475],
        [0.2602],
        [0.2730],
        [0.2732],
        [0.1522],
        [0.2707],
        [0.3727],
        [0.3549],
        [0.3379],
        [0.2935],
        [0.3549],
        [0.2474],
        [0.2812],
        [0.2668],
        [0.2524],
        [0.2818],
        [0.2674],
        [0.2150],
        [0.3240],
        [0.3180],
        [0.2201],
        [0.2305],
        [0.3067],
        [0.2937],
        [0.3573],
        [0

In [41]:
from sklearn.metrics import mean_squared_error

In [43]:
r2_score(testpredict.detach().cpu().numpy(),Y_test)

-58.924012065645286

In [45]:
mean_squared_error(testpredict.detach().cpu().numpy(),Y_test)

0.13273072330622904

In [ ]:
import math

In [ ]:
def average(x):
    assert len(x) > 0
    return float(sum(x)) / len(x)

def pearson_def(x, y):
    assert len(x) == len(y)
    n = len(x)
    assert n > 0
    avg_x = average(x)
    avg_y = average(y)
    diffprod = 0
    xdiff2 = 0
    ydiff2 = 0
    for idx in range(n):
        xdiff = x[idx] - avg_x
        ydiff = y[idx] - avg_y
        diffprod += xdiff * ydiff
        xdiff2 += xdiff * xdiff
        ydiff2 += ydiff * ydiff

    return diffprod / math.sqrt(xdiff2 * ydiff2)

In [ ]:
print pearson_def([1,2,3],[4,5,6])

In [ ]:
testpredict.detach().numpy()